# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_key import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data = pd.read_csv("../output_data/weather_data.csv")
weather_data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Cape Town,-33.9258,18.4232,22.00,49,40,6.17,ZA,1613903406
1,Safaga,26.7292,33.9365,19.00,34,0,4.12,EG,1613903603
2,Atuona,-9.8000,-139.0333,26.33,79,85,8.73,PF,1613903603
3,Faya,18.3851,42.4509,18.00,67,40,9.77,SA,1613903603
4,Bluff,-46.6000,168.3333,15.00,84,0,0.89,NZ,1613903603
...,...,...,...,...,...,...,...,...,...
543,Rumphi,-11.0186,33.8575,26.55,71,80,0.40,MW,1613903723
544,Togur,58.3631,82.8265,-18.82,92,100,4.27,RU,1613903723
545,Krasnokholmskiy,55.9873,55.0466,-19.30,91,100,2.96,RU,1613903724
546,Saint-François,46.4154,3.9054,16.67,44,0,10.80,FR,1613903724


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store lat, lng and Humidity in variables 
locations = weather_data[["Lat", "Lng"]]
weights = weather_data["Humidity"]

In [10]:
# Plot Heatmap
figure_layout = {
    'width': '900px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout, center=(34.0, -118.0), zoom_level=2)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights, dissipating=False, max_intensity=100, point_radius=3)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [23]:
# Set weather condition to be -> wind speed greater than 5, humidity greater than 50 and max temp between 25 and 50
narrowed_cities_df = weather_data.loc[(weather_data["Wind Speed"] >= 5) & (weather_data["Humidity"] > 50) &
                                   (weather_data["Max Temp"] >= 25) & (weather_data["Max Temp"] <= 50)].dropna()

narrowed_cities_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,Atuona,-9.8000,-139.0333,26.33,79,85,8.73,PF,1613903603
12,Marabu,4.4148,114.0089,27.00,78,20,5.66,MY,1613903605
44,Itarema,-2.9248,-39.9167,27.41,74,89,6.05,BR,1613903612
45,Saint-Philippe,-21.3585,55.7679,32.00,62,20,8.23,RE,1613903612
46,Vaini,-21.2000,-175.2000,25.00,73,75,5.66,TO,1613903612


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [25]:
hotel_df = narrowed_cities_df.loc[:,["Country", "City", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Display the result
hotel_df.head()

,Country,City,Lat,Lng,Hotel Name
2,PF,Atuona,-9.8000,-139.0333,
12,MY,Marabu,4.4148,114.0089,
44,BR,Itarema,-2.9248,-39.9167,
45,RE,Saint-Philippe,-21.3585,55.7679,
46,TO,Vaini,-21.2000,-175.2000,


In [42]:
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key,
            }

In [51]:
for index, row in hotel_df.iterrows():

    lat = row["Lat"]
    lng = row["Lng"]
    cities = row["City"]
    params["location"] = f"{lat},{lng}"

    # Make API request and extract request results
    response = requests.get(url, params=params).json()
    results = response["results"]

    # Add retrieved results to data frame
    try:
        print(f"Found the closest hotel in {cities}: {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    except (KeyError, IndexError):
        print("Results not found. Skipping...")
        
    print("------------")
    

# Print end of search once searching is completed
print("Data Retrieval Complete")

Found the closest hotel in Atuona: Hôtel Hiva Oa Hanakee Lodge.
------------
Found the closest hotel in Marabu: Miri Marriott Resort & Spa.
------------
Found the closest hotel in Itarema: Pousada Villa Del Lótus.
------------
Found the closest hotel in Saint-Philippe: Les Embruns Du Baril.
------------
Found the closest hotel in Vaini: Paradise First Hotel.
------------
Found the closest hotel in Hithadhoo: Abuharee Grand.
------------
Found the closest hotel in Bandarbeyla: JABIR HOTEL.
------------
Results not found. Skipping...
------------
Found the closest hotel in Karratha: ibis Styles Karratha.
------------
Found the closest hotel in Alofi: Taloa Heights.
------------
Found the closest hotel in Geraldton: Ocean Centre Hotel.
------------
Results not found. Skipping...
------------
Found the closest hotel in Le Port: Lodge Roche Tamarin & Spa Village Nature.
------------
Found the closest hotel in Abu Dhabi: Emirates Palace.
------------
Found the closest hotel in Hobyo: Osmani 

In [53]:
hotel_df.head()

,Country,City,Lat,Lng,Hotel Name
2,PF,Atuona,-9.8000,-139.0333,Hôtel Hiva Oa Hanakee Lodge
12,MY,Marabu,4.4148,114.0089,Miri Marriott Resort & Spa
44,BR,Itarema,-2.9248,-39.9167,Pousada Villa Del Lótus
45,RE,Saint-Philippe,-21.3585,55.7679,Les Embruns Du Baril
46,TO,Vaini,-21.2000,-175.2000,Paradise First Hotel


In [54]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [55]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…